In [3]:
from nltk.stem import PorterStemmer
import os
import string
import pandas as pd
from collections import Counter
import math
from operator import itemgetter
import time
from collections import OrderedDict
import itertools
start_time = time.time()

In [108]:
def createStopWords(filename):
    stopWords = []
    with open(filename, "r") as file_object:
        for line in file_object:
            stopWords.append(line.rstrip()) 
    return stopWords

def readFilesAndReturnCleanList(stopWords):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~/+-1234567890'''
    cleanDocuments = {}
    individualDocument = []
    for file in os.listdir('data/'):
        if file.endswith(".txt"):
            documentName = os.path.join(file)
            with open("data/" + documentName, "r") as document:
                for line in document:
                    cleanLine = line.rstrip()
                    listofWordsInALine = cleanLine.split(" ")
                    cleanWord = ""
                    ps = PorterStemmer()
                    for word in listofWordsInALine:
                        if word.lower() not in stopWords:
                            word = ps.stem(word)
                            for letter in word:
                                if letter not in punctuations:
                                    cleanWord = cleanWord + letter
                            if (cleanWord != ""):
                                individualDocument.append(cleanWord)             
                            cleanWord = ""
                cleanDocuments[documentName] = individualDocument 
                individualDocument = [] 
    return cleanDocuments  

def createUniqueWordsList(documentsInADictionary):
    listOfUniqueWords = []
    for document, words in documentsInADictionary.items():
        for word in words:
            if word not in listOfUniqueWords:
                listOfUniqueWords.append(word)
    return listOfUniqueWords

def termFrequency(documentName, document):
    lengthOfDocument = len(document)
    wordInformation = []
    wordInfoDictionary = {} 
    documentAndWordInfo = {}
    wordCounter = Counter(document)
    if lengthOfDocument != 0:
        for word, repetation in wordCounter.items():
            frequency = repetation / lengthOfDocument
            # wordInformation.append(repetation)
            wordInformation.append(frequency)
            # wordInformation.append(lengthOfDocument)

            wordInfoDictionary[word] = wordInformation
            wordInformation = [] 
    documentAndWordInfo[documentName] = wordInfoDictionary
    return documentAndWordInfo

def findTermFrequency(documentsInADictionary):
    tf = []
    for documentName, document in documentsInADictionary.items():
        documentAndWordInfo = termFrequency(documentName, document)
        tf.append(documentAndWordInfo)
    return tf

def findInverseDocumentFrequency(listOfUniqueWords, documentsInADictionary, tf):
    idf = {}
    numberOfDocuments = len(documentsInADictionary)
    for word in listOfUniqueWords:
        for document, words in documentsInADictionary.items():
            if word in words:
                if word in idf:
                    idf[word] =  idf[word] + 1
                else:
                    idf[word] = 1
    for word, repetation in idf.items():
        idf[word] = math.log10(numberOfDocuments / repetation)

    return idf

def tfIdf(tf, idf):
    tfIdfReturnDictionary = {}
    listForWordAndValue = []
    mainDocumentList = []

    for documentAndWords in tf:
        documentName = ""
        for document, wordsInfo in documentAndWords.items():
            documentName = document
            if len(wordsInfo) != 0:
                for word, wordTf in wordsInfo.items():
                    tfIdfValue = wordTf[0] * idf[word]

                    listForWordAndValue.append(word)
                    listForWordAndValue.append(tfIdfValue)
                    mainDocumentList.append(listForWordAndValue)
                    listForWordAndValue = []
        mainDocumentList = sorted(mainDocumentList, key=itemgetter(1), reverse=True)
        tfIdfReturnDictionary[documentName] = mainDocumentList
        documentName = ""
        mainDocumentList = []
    return tfIdfReturnDictionary

def createInputFileForAprioriAlgorithm(tf_idf, topN):
    count = 0
    writeToFile = open("aprioriInput.txt", "w") 
    for documentName, values in tf_idf.items():
        length = len(values[:topN])
        if len(values) != 0:
            for i in values[:topN]:
                count = count + 1
                if count == length:
                    writeToFile.write(i[0])
                else:
                    writeToFile.write(i[0] + ", ")
            count = 0
            writeToFile.write("\n")

def generateC1(dataSet):
    productDict = {}
    returneSet = []
    for data in dataSet:
        for product in data:
            if product not in productDict:
               productDict[product] = 1
            else:
                 productDict[product] = productDict[product] + 1
    for key in productDict:
        tempArray = []
        tempArray.append(key)
        returneSet.append(tempArray)
        returneSet.append(productDict[key])
        tempArray = []
    return returneSet

def generateFrequentItemSet(CandidateList, noOfTransactions, minimumSupport, dataSet, fatherFrequentArray):
    frequentItemsArray = []
    for i in range(len(CandidateList)):
        if i%2 != 0:
            support = (CandidateList[i] * 1.0 / noOfTransactions) * 100
            if support >= minimumSupport:
                frequentItemsArray.append(CandidateList[i-1])
                frequentItemsArray.append(CandidateList[i])
            else:
                eleminatedItemsArray.append(CandidateList[i-1])

    for k in frequentItemsArray:
        fatherFrequentArray.append(k)

    if len(frequentItemsArray) == 2 or len(frequentItemsArray) == 0:
        #print("This will be returned")
        returnArray = fatherFrequentArray
        return returnArray

    else:
        generateCandidateSets(dataSet, eleminatedItemsArray, frequentItemsArray, noOfTransactions, minimumSupport)
        
def generateCandidateSets(dataSet, eleminatedItemsArray, frequentItemsArray, noOfTransactions, minimumSupport):
    onlyElements = []
    arrayAfterCombinations = []
    candidateSetArray = []
    for i in range(len(frequentItemsArray)):
        if i%2 == 0:
            onlyElements.append(frequentItemsArray[i])
    for item in onlyElements:
        tempCombinationArray = []
        k = onlyElements.index(item)
        for i in range(k + 1, len(onlyElements)):
            for j in item:
                if j not in tempCombinationArray:
                    tempCombinationArray.append(j)
            for m in onlyElements[i]:
                if m not in tempCombinationArray:
                    tempCombinationArray.append(m)
            arrayAfterCombinations.append(tempCombinationArray)
            tempCombinationArray = []
    sortedCombinationArray = []
    uniqueCombinationArray = []
    for i in arrayAfterCombinations:
        sortedCombinationArray.append(sorted(i))
    for i in sortedCombinationArray:
        if i not in uniqueCombinationArray:
            uniqueCombinationArray.append(i)
    arrayAfterCombinations = uniqueCombinationArray
    for item in arrayAfterCombinations:
        count = 0
        for transaction in dataSet:
            if set(item).issubset(set(transaction)):
                count = count + 1
        if count != 0:
            candidateSetArray.append(item)
            candidateSetArray.append(count)
    generateFrequentItemSet(candidateSetArray, noOfTransactions, minimumSupport, dataSet, fatherFrequentArray)

In [109]:
stopWords = createStopWords("listOfStopWords.txt")
documentsInADictionary = readFilesAndReturnCleanList(stopWords)

In [110]:
listOfUniqueWords = createUniqueWordsList(documentsInADictionary)

In [111]:
tf = findTermFrequency(documentsInADictionary)
idf = findInverseDocumentFrequency(listOfUniqueWords, documentsInADictionary, tf)

In [112]:
tf_idf = tfIdf(tf, idf)

In [113]:
topN = input("Enter value of N: ")
topN = int(topN)

Enter value of N: 1500


In [114]:
createInputFileForAprioriAlgorithm(tf_idf, topN)

In [115]:
minimumSupport = input('Enter minimum Support: ')
minimumConfidence = input('Enter minimum Confidence: ')

minimumSupport = int(minimumSupport)
minimumConfidence = int(minimumConfidence)

# minimumSupport = 10
# minimumConfidence = 10
nonFrequentSets = []
allFrequentItemSets = []
tempFrequentItemSets = []
dataSet = []
eleminatedItemsArray = []
noOfTransactions = 1
fatherFrequentArray = []


Enter minimum Support: 10
Enter minimum Confidence: 10


In [116]:
with open("aprioriInput.txt") as fp:
    lines = fp.readlines()

for line in lines:
    line = line.rstrip()
    dataSet.append(line.split(","))

In [117]:
firstCandidateSet = generateC1(dataSet)

In [ ]:
frequentItemSet = generateFrequentItemSet(firstCandidateSet, noOfTransactions, minimumSupport, dataSet, fatherFrequentArray)

In [ ]:
frequentItemSet

In [67]:
from nltk.corpus import stopwords
import pandas as pd
from string import punctuation
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
lemmatizer = WordNetLemmatizer() 

df_1=pd.read_csv('5_threat_groups.csv')

In [56]:
stop_words=stopwords.words("english")+list(punctuation)
def tokenization(text):
    pre_processed_text=[]
    words = [w for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)]    
    tokens=[w for w in words if w.lower() not in stop_words] 
    tokens=[w.lower() for w in tokens if ((w.isalnum()) & (len(w)>1))]
    tokens=[lemmatizer.lemmatize(w, pos ="v") for w in tokens]
    return tokens

In [68]:
docs=[]
col=df_1.columns
for i in col:
    text=df_1[i][0]
    tokens= tokenization(text)
    docs.append(' '.join(tokens))

In [69]:
cv=CountVectorizer()
word_count_vector=cv.fit_transform(docs)

In [72]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [73]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights'])

,idf_weights
documentation,1.000000
time,1.000000
describe,1.000000
available,1.000000
15,1.000000
...,...
b42,2.098612
jury,2.098612
jurisdiction,2.098612
b2f6d9a62c63f61a6b33dc6520bfcccd,2.098612


In [74]:
count_vector=cv.transform(docs)
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [77]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
apt1,0.638520
use,0.212312
shanghai,0.168401
unit,0.156247
61398,0.156122
...,...
john356gh,0.000000
john,0.000000
joeuchill,0.000000
audiwheel,0.000000
